
# Calculating Probability and Proportion

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

c:\Users\Chris Lowson\new.conda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
flights = pd.read_csv('flights_clean.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'flights_clean.csv'

In [286]:
flights
#A negative number means the flight arrived early

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
0,2015,1,1,4,AS,98,ANC,SEA,5,2354.0,-11.0,205.0,194.0,430,408.0,-22.0,0,0
1,2015,1,1,4,AA,2336,LAX,PBI,10,2.0,-8.0,280.0,279.0,750,741.0,-9.0,0,0
2,2015,1,1,4,US,840,SFO,CLT,20,18.0,-2.0,286.0,293.0,806,811.0,5.0,0,0
3,2015,1,1,4,AA,258,LAX,MIA,20,15.0,-5.0,285.0,281.0,805,756.0,-9.0,0,0
4,2015,1,1,4,AS,135,SEA,ANC,25,24.0,-1.0,235.0,215.0,320,259.0,-21.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245479,2015,12,31,4,B6,688,LAX,BOS,2359,2355.0,-4.0,320.0,298.0,819,753.0,-26.0,0,0
5245480,2015,12,31,4,B6,745,JFK,PSE,2359,2355.0,-4.0,227.0,215.0,446,430.0,-16.0,0,0
5245481,2015,12,31,4,B6,1503,JFK,SJU,2359,2350.0,-9.0,221.0,222.0,440,432.0,-8.0,0,0
5245482,2015,12,31,4,B6,333,MCO,SJU,2359,2353.0,-6.0,161.0,157.0,340,330.0,-10.0,0,0


In [158]:
flights.shape
#having 5,245,484 rows is what I am going to start as my asumption as total flights

(5245484, 18)

In [159]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5245484 entries, 0 to 5245483
Data columns (total 18 columns):
 #   Column               Dtype  
---  ------               -----  
 0   year                 int64  
 1   month                int64  
 2   day                  int64  
 3   day_of_week          int64  
 4   airline              object 
 5   flight_number        int64  
 6   origin_airport       object 
 7   destination_airport  object 
 8   scheduled_departure  int64  
 9   departure_time       float64
 10  departure_delay      float64
 11  scheduled_time       float64
 12  elapsed_time         float64
 13  scheduled_arrival    int64  
 14  arrival_time         float64
 15  arrival_delay        float64
 16  diverted             int64  
 17  cancelled            int64  
dtypes: float64(6), int64(9), object(3)
memory usage: 720.4+ MB


In [161]:
flights.groupby('airline').size().sort_values(ascending=False)

airline
WN    1141618
DL     796522
AA     638500
OO     529826
EV     511495
UA     463384
MQ     257898
B6     240989
US     194648
AS     157418
NK     105228
F9      82159
HA      69871
VX      55928
dtype: int64

WN is the most popular airline by far! (South West)

In [162]:
flights.groupby('origin_airport').size().sort_values(ascending=False)

origin_airport
ATL    344279
ORD    277336
DFW    233297
DEN    193932
LAX    192509
        ...  
STC        78
DLG        77
GST        76
AKN        63
ITH        30
Length: 322, dtype: int64

In [163]:
flights.groupby('destination_airport').size().sort_values(ascending=False)

destination_airport
ATL    344189
ORD    276633
DFW    232833
DEN    193700
LAX    192437
        ...  
STC        77
DLG        77
GST        76
AKN        63
ITH        31
Length: 322, dtype: int64

In [164]:
flights_late = flights['arrival_delay'].mask(flights['arrival_delay'] > 0, 'yes')
flights_late.value_counts()
# I made a mask to discover how many delayed flights exist, and value over 0 would be a "yes" = 1,947,990 
## I did this as an initial check and decided to not use this cell for anlysis but left it here for visability and a second check to my count.

yes        1947990
-8.0        160199
-9.0        159394
-10.0       158512
-7.0        158445
            ...   
-150.35          1
-48.6            1
-149.05          1
-82.9            1
-245.6           1
Name: arrival_delay, Length: 1755, dtype: int64

1.9 Million total delayed flights from this data and the most common delay is about 8 or 9 minutes. ATL is the most trafficked airport! 

In [165]:
flights_delayed = flights[flights['arrival_delay'] >0]
flights_delayed #again number of flights delayed but now its own df 

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
2,2015,1,1,4,US,840,SFO,CLT,20,18.0,-2.0,286.0,293.0,806,811.0,5.0,0,0
5,2015,1,1,4,DL,806,SFO,MSP,25,20.0,-5.0,217.0,230.0,602,610.0,8.0,0,0
14,2015,1,1,4,DL,2440,SEA,MSP,40,39.0,-1.0,189.0,198.0,549,557.0,8.0,0,0
20,2015,1,1,4,NK,520,LAS,MCI,55,120.0,25.0,162.0,143.0,537,543.0,6.0,0,0
21,2015,1,1,4,AA,371,SEA,MIA,100,52.0,-8.0,338.0,347.0,938,939.0,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245462,2015,12,31,4,NK,371,FLL,BQN,2358,11.0,13.0,140.0,155.0,318,346.0,28.0,0,0
5245474,2015,12,31,4,B6,1248,LAS,JFK,2359,238.0,159.0,282.0,282.0,741,1020.0,159.0,0,0
5245476,2015,12,31,4,B6,802,SLC,MCO,2359,15.0,16.0,249.0,250.0,608,625.0,17.0,0,0
5245478,2015,12,31,4,B6,66,ABQ,JFK,2359,15.0,16.0,227.0,214.0,546,549.0,3.0,0,0


This was a secondary way to show that there were 1.95 million flights delayed in total, I can represent the alternative below.

In [166]:
flights_early = flights[flights['arrival_delay'] <=0]
flights_early #own df for number of flights early or on time 

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
0,2015,1,1,4,AS,98,ANC,SEA,5,2354.0,-11.0,205.0,194.0,430,408.0,-22.0,0,0
1,2015,1,1,4,AA,2336,LAX,PBI,10,2.0,-8.0,280.0,279.0,750,741.0,-9.0,0,0
3,2015,1,1,4,AA,258,LAX,MIA,20,15.0,-5.0,285.0,281.0,805,756.0,-9.0,0,0
4,2015,1,1,4,AS,135,SEA,ANC,25,24.0,-1.0,235.0,215.0,320,259.0,-21.0,0,0
6,2015,1,1,4,NK,612,LAS,MSP,25,19.0,-6.0,181.0,170.0,526,509.0,-17.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245477,2015,12,31,4,B6,98,DEN,JFK,2359,6.0,7.0,211.0,193.0,530,519.0,-11.0,0,0
5245479,2015,12,31,4,B6,688,LAX,BOS,2359,2355.0,-4.0,320.0,298.0,819,753.0,-26.0,0,0
5245480,2015,12,31,4,B6,745,JFK,PSE,2359,2355.0,-4.0,227.0,215.0,446,430.0,-16.0,0,0
5245481,2015,12,31,4,B6,1503,JFK,SJU,2359,2350.0,-9.0,221.0,222.0,440,432.0,-8.0,0,0


3.2M flights actually arrived on or ahead of time to their destination

In [167]:
flights_total = flights.groupby('origin_airport').size().sort_values(ascending=False)
flights_total 
#total flights formatted to calculate proportion 

origin_airport
ATL    344279
ORD    277336
DFW    233297
DEN    193932
LAX    192509
        ...  
STC        78
DLG        77
GST        76
AKN        63
ITH        30
Length: 322, dtype: int64

In [168]:
delay_yes = flights_delayed.groupby('origin_airport').size().sort_values(ascending=False)
delay_yes 
# number of flights arriving delayed by origin airport 

origin_airport
ATL    115034
ORD    113413
DFW     93195
DEN     80870
LAX     79754
        ...  
VEL        23
HYA        21
DLG        17
ITH        14
CNY         8
Length: 322, dtype: int64

In [169]:
delay_no = flights_early.groupby('origin_airport').size().sort_values(ascending=False)
delay_no 
# number of flights not arriving delayed by origin airport

origin_airport
ATL    229245
ORD    163923
DFW    140102
DEN    113062
LAX    112755
        ...  
PPG        40
ADK        32
AKN        31
GST        25
ITH        16
Length: 322, dtype: int64

In [170]:
prop_early = delay_no/flights_total
prop_early.sort_values(ascending=False).head(20) #proportion a flight from origin airport will be early or on time 

origin_airport
CNY    0.960976
VEL    0.885000
BTM    0.847222
LWS    0.835034
DIK    0.792617
EKO    0.783366
DLG    0.779221
BIL    0.778128
GCC    0.776876
ESC    0.773381
DAB    0.773154
CPR    0.772493
RKS    0.772321
ECP    0.767414
HLN    0.763675
LAR    0.763016
IDA    0.757630
TOL    0.754738
RDD    0.753541
YAK    0.753425
dtype: float64

In [223]:
prop_delay = delay_yes/flights_total
prop_delay.sort_values(ascending=False).head(20) #proportion a flight from origin airport will be delayed

origin_airport
GST    0.671053
ADK    0.640449
PPG    0.626168
ILG    0.557895
AKN    0.507937
RHI    0.489883
OME    0.488994
STC    0.487179
COD    0.479326
DLH    0.466707
ITH    0.466667
OTZ    0.466459
PBG    0.465950
DAL    0.452446
CIU    0.450586
BPT    0.447253
ASE    0.445526
CRW    0.444540
OAK    0.441248
BTR    0.439383
dtype: float64

Which airport(s) had the largest proportion of flights arriving late at their destination airport?: Leaving from airport GST gives you a 67% chance your flight will be delayed based off these proportions! 

the airport(s) with the lowest proportion of late arriving flights at their destination airport?: Leaving from airport CNY youre almost guarunteed to land early of on time witha 96% chance of landing with no delay based off these proportions! 

In [172]:
flights.groupby('destination_airport').size().sort_values(ascending=False) 

destination_airport
ATL    344189
ORD    276633
DFW    232833
DEN    193700
LAX    192437
        ...  
STC        77
DLG        77
GST        76
AKN        63
ITH        31
Length: 322, dtype: int64

In [237]:
prop_delay.sort_values(ascending=False)

origin_airport
GST    0.671053
ADK    0.640449
PPG    0.626168
ILG    0.557895
AKN    0.507937
         ...   
DIK    0.207383
LWS    0.164966
BTM    0.152778
VEL    0.115000
CNY    0.039024
Length: 322, dtype: float64

In [235]:
prop_delay['ATL']

0.33413016768376813

The proportion of delayed flights leaving each airport. I wanted to see ATL specifically since it is the most trafficked

In [231]:
origin_yes = flights_delayed.groupby('origin_airport').size().sort_values(ascending=False)
origin_yes

origin_airport
ATL    115034
ORD    113413
DFW     93195
DEN     80870
LAX     79754
        ...  
VEL        23
HYA        21
DLG        17
ITH        14
CNY         8
Length: 322, dtype: int64

In [208]:
destination_yes = flights_delayed.groupby('destination_airport').size().sort_values(ascending=False)
destination_yes

destination_airport
ATL    105696
ORD    104305
DFW     83515
LAX     82048
DEN     71510
        ...  
CNY        27
AKN        26
VEL        19
DLG        17
ITH        13
Length: 322, dtype: int64

In [221]:
total = flights.shape[0]
total

5245484

In [227]:
f_d = flights_delayed.shape[0]
f_d

1947990

In [233]:
orig_prob = origin_yes.apply(lambda x: x/total)
orig_prob.sort_values(ascending=False).head(20) #probability of delay departing an airport 

origin_airport
ATL    0.021930
ORD    0.021621
DFW    0.017767
DEN    0.015417
LAX    0.015204
IAH    0.011420
PHX    0.011125
SFO    0.010722
LAS    0.010095
SEA    0.008311
CLT    0.007908
MCO    0.007889
BOS    0.007401
MSP    0.007167
DTW    0.006925
LGA    0.006916
EWR    0.006873
BWI    0.006675
JFK    0.006391
MDW    0.006082
dtype: float64

If you fly from ATL to any given airport theres a 2.19% chance you'll arrive delayed 

In [230]:
delayed_prob = destination_yes.apply(lambda x: x/total)
delayed_prob.sort_values(ascending=False).head(20) 
#probability of delay flying into airport 

destination_airport
ATL    0.020150
ORD    0.019885
DFW    0.015921
LAX    0.015642
DEN    0.013633
SFO    0.010656
PHX    0.010419
IAH    0.010276
LAS    0.009257
MCO    0.007945
SEA    0.007715
LGA    0.007708
BOS    0.007701
MSP    0.007574
EWR    0.007122
DTW    0.006988
CLT    0.006823
JFK    0.006459
BWI    0.005751
SLC    0.005533
dtype: float64

This is the overall likelhood or probability that your flight will be delayed flying into any given airport. if you fly into ATL you have a 2.01% chance of being delayed

In [234]:
prob_any_delay = f_d/total
prob_any_delay #probability of expereincing a delay flying to or from any airport

0.3713651590587256

from this dataset we know the amount of delays was 1.9M and the amount of total of flights is 5.2M 
So the P(delayed) is = Occurances of Delays / total possible occurances = .3713 or 37.16% 

Bernoullis distribution:

I have just discovered that the probability of any given flight being delayed is 37%, I would like to know what chance there is if I take 100 flights that I will be delayed at least 25 times

In [357]:
stats.binom.pmf(25, n=100, p=.37)

0.0034700086907805535

The possibility I am delayed at least 25 times from 100 unique flights going to or from any given airport is about 3.5%, based on the fact that the overall chance of being delayed is 37% 

TSA Timeliness! 

Lets say I am going to be flying from Southern California into Northern California. I live equidistant from the two souther california airports and will be traveling in the bay area equidistant from the two bay area airports. This will be a round trip flight and I have the option to mix and match airports. I want to know the airports in both Northern and Southern California with that have the best proportion of flights leaving and departing early. Assuming cost of flight is the same regardless of which airport I choose, this is how I will decide.  

In [295]:
LAX_origin = flights.loc[flights['origin_airport'] == 'LAX']
LAX_origin #Los Angeles

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
1,2015,1,1,4,AA,2336,LAX,PBI,10,2.0,-8.0,280.0,279.0,750,741.0,-9.0,0,0
3,2015,1,1,4,AA,258,LAX,MIA,20,15.0,-5.0,285.0,281.0,805,756.0,-9.0,0,0
7,2015,1,1,4,US,2013,LAX,CLT,30,44.0,14.0,273.0,249.0,803,753.0,-10.0,0,0
12,2015,1,1,4,DL,1434,LAX,MSP,35,35.0,0.0,214.0,210.0,609,605.0,-4.0,0,0
23,2015,1,1,4,AA,115,LAX,MIA,105,103.0,-2.0,286.0,276.0,851,839.0,-12.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245454,2015,12,31,4,AA,1538,LAX,MIA,2355,2357.0,2.0,293.0,274.0,748,731.0,-17.0,0,0
5245459,2015,12,31,4,AS,175,LAX,ANC,2355,2350.0,-5.0,350.0,310.0,445,400.0,-45.0,0,0
5245460,2015,12,31,4,AS,471,LAX,SEA,2355,2353.0,-2.0,169.0,152.0,244,225.0,-19.0,0,0
5245473,2015,12,31,4,AA,219,LAX,ORD,2359,2358.0,-1.0,244.0,226.0,603,544.0,-19.0,0,0


In [325]:
SAN_origin = flights.loc[flights['origin_airport'] == 'SAN']
SAN_origin #San Diego 

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
432,2015,1,1,4,DL,978,SAN,SLC,615,611.0,-4.0,121.0,115.0,916,906.0,-10.0,0,0
453,2015,1,1,4,OO,5608,SAN,LAX,615,614.0,-1.0,52.0,48.0,707,702.0,-5.0,0,0
470,2015,1,1,4,WN,823,SAN,BWI,620,620.0,0.0,295.0,272.0,1415,1352.0,-23.0,0,0
471,2015,1,1,4,WN,603,SAN,MDW,620,617.0,-3.0,250.0,224.0,1230,1201.0,-29.0,0,0
473,2015,1,1,4,UA,1192,SAN,DEN,620,618.0,-2.0,145.0,138.0,945,936.0,-9.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245218,2015,12,31,4,B6,20,SAN,BOS,2200,2150.0,-10.0,309.0,283.0,609,533.0,-36.0,0,0
5245222,2015,12,31,4,UA,1916,SAN,EWR,2202,2201.0,-1.0,298.0,276.0,600,537.0,-23.0,0,0
5245288,2015,12,31,4,DL,1792,SAN,ATL,2230,2223.0,-7.0,234.0,228.0,524,511.0,-13.0,0,0
5245304,2015,12,31,4,UA,240,SAN,ORD,2235,2240.0,5.0,248.0,205.0,443,405.0,-38.0,0,0


In [324]:
SFO_origin = flights.loc[flights['origin_airport'] == 'SFO']
SFO_origin #San Francisco 

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
2,2015,1,1,4,US,840,SFO,CLT,20,18.0,-2.0,286.0,293.0,806,811.0,5.0,0,0
5,2015,1,1,4,DL,806,SFO,MSP,25,20.0,-5.0,217.0,230.0,602,610.0,8.0,0,0
8,2015,1,1,4,AA,1112,SFO,DFW,30,19.0,-11.0,195.0,193.0,545,532.0,-13.0,0,0
17,2015,1,1,4,UA,1197,SFO,IAH,48,42.0,-6.0,218.0,217.0,626,619.0,-7.0,0,0
87,2015,1,1,4,UA,1532,SFO,DEN,531,550.0,19.0,146.0,153.0,857,923.0,26.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245405,2015,12,31,4,OO,6220,SFO,RNO,2325,2309.0,-16.0,64.0,52.0,29,1.0,-28.0,0,0
5245410,2015,12,31,4,OO,6376,SFO,MRY,2330,2319.0,-11.0,44.0,37.0,14,2356.0,-18.0,0,0
5245412,2015,12,31,4,UA,342,SFO,ATL,2330,2326.0,-4.0,279.0,284.0,709,710.0,1.0,0,0
5245429,2015,12,31,4,UA,1204,SFO,IAH,2343,2337.0,-6.0,222.0,202.0,525,459.0,-26.0,0,0


In [350]:
SJC_origin = flights.loc[flights['origin_airport'] == 'SJC']
SJC_origin #San Jose

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
385,2015,1,1,4,OO,4494,SJC,SLC,609,559.0,-10.0,122.0,144.0,911,923.0,12.0,0,0
475,2015,1,1,4,US,460,SJC,PHX,620,614.0,-6.0,113.0,110.0,913,904.0,-9.0,0,0
603,2015,1,1,4,DL,1056,SJC,MSP,639,634.0,-5.0,231.0,234.0,1230,1228.0,-2.0,0,0
613,2015,1,1,4,WN,832,SJC,LAS,640,640.0,0.0,85.0,80.0,805,800.0,-5.0,0,0
703,2015,1,1,4,AS,321,SJC,SEA,650,647.0,-3.0,124.0,113.0,854,840.0,-14.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5243862,2015,12,31,4,WN,5607,SJC,SAN,1840,1829.0,-11.0,75.0,71.0,1955,1940.0,-15.0,0,0
5243982,2015,12,31,4,AS,319,SJC,SEA,1850,1836.0,-14.0,130.0,124.0,2100,2040.0,-20.0,0,0
5244209,2015,12,31,4,AA,411,SJC,PHX,1917,1858.0,-19.0,103.0,97.0,2200,2135.0,-25.0,0,0
5245348,2015,12,31,4,DL,1684,SJC,ATL,2255,2253.0,-2.0,262.0,247.0,617,600.0,-17.0,0,0


Prop_early represents the earlier calculation of the proportion of flights that arrived early or on time to these respective airports.
Prop_delay would be the alternative count of how many flights arrived delayed at these respective airports 

In [316]:
prop_early['LAX']

0.5857128757616527

In [318]:
prop_early['SAN']

0.6385583820690848

In [319]:
prop_early['SFO']

0.6144179350061703

In [346]:
prop_early['SJC']

0.60102748795085

In [317]:
prop_delay['LAX']

0.4142871242383473

In [320]:
prop_delay['SAN']

0.3614416179309153

In [347]:
prop_delay['SJC']

0.39897251204914996

In [321]:
prop_delay['SFO']

0.3855820649938297

From the four tables I made a few cells above I can first see which airport has the most air traffic, I wamt to assume the more planes going in and out the higher chance of delays. From this data set SAN and SJC are by far the less trafficked airports, LAX being the most busy. What I was able to see just above was that SAN and SFO have the higher proportion of planes arriving at their gates early or on time (surprising to see its SFO, the much busier airport) I may want to consider flying into SFO and back into SAN 

In [330]:
departure_analysis_y = flights[flights['departure_delay'] >0]
departure_analysis_y

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
7,2015,1,1,4,US,2013,LAX,CLT,30,44.0,14.0,273.0,249.0,803,753.0,-10.0,0,0
9,2015,1,1,4,DL,1173,LAS,ATL,30,33.0,3.0,221.0,203.0,711,656.0,-15.0,0,0
20,2015,1,1,4,NK,520,LAS,MCI,55,120.0,25.0,162.0,143.0,537,543.0,6.0,0,0
27,2015,1,1,4,NK,597,MSP,FLL,115,127.0,12.0,207.0,220.0,542,607.0,25.0,0,0
29,2015,1,1,4,AA,2392,DEN,MIA,120,141.0,21.0,227.0,208.0,707,709.0,2.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245474,2015,12,31,4,B6,1248,LAS,JFK,2359,238.0,159.0,282.0,282.0,741,1020.0,159.0,0,0
5245476,2015,12,31,4,B6,802,SLC,MCO,2359,15.0,16.0,249.0,250.0,608,625.0,17.0,0,0
5245477,2015,12,31,4,B6,98,DEN,JFK,2359,6.0,7.0,211.0,193.0,530,519.0,-11.0,0,0
5245478,2015,12,31,4,B6,66,ABQ,JFK,2359,15.0,16.0,227.0,214.0,546,549.0,3.0,0,0


In [327]:
departure_analysis_n = flights[flights['departure_delay'] <=0]
departure_analysis_n

,year,month,day,day_of_week,airline,flight_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,scheduled_time,elapsed_time,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled
0,2015,1,1,4,AS,98,ANC,SEA,5,2354.0,-11.0,205.0,194.0,430,408.0,-22.0,0,0
1,2015,1,1,4,AA,2336,LAX,PBI,10,2.0,-8.0,280.0,279.0,750,741.0,-9.0,0,0
2,2015,1,1,4,US,840,SFO,CLT,20,18.0,-2.0,286.0,293.0,806,811.0,5.0,0,0
3,2015,1,1,4,AA,258,LAX,MIA,20,15.0,-5.0,285.0,281.0,805,756.0,-9.0,0,0
4,2015,1,1,4,AS,135,SEA,ANC,25,24.0,-1.0,235.0,215.0,320,259.0,-21.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245475,2015,12,31,4,B6,80,RNO,JFK,2359,2359.0,0.0,306.0,285.0,805,744.0,-21.0,0,0
5245479,2015,12,31,4,B6,688,LAX,BOS,2359,2355.0,-4.0,320.0,298.0,819,753.0,-26.0,0,0
5245480,2015,12,31,4,B6,745,JFK,PSE,2359,2355.0,-4.0,227.0,215.0,446,430.0,-16.0,0,0
5245481,2015,12,31,4,B6,1503,JFK,SJU,2359,2350.0,-9.0,221.0,222.0,440,432.0,-8.0,0,0


In [331]:
dep_del_yes = departure_analysis_y.groupby('origin_airport').size().sort_values(ascending=False)
dep_del_yes

origin_airport
ATL    129782
ORD    121380
DFW     96189
DEN     89183
LAX     81914
        ...  
STC        28
DLG        25
CNY        11
ITH        11
VEL        10
Length: 322, dtype: int64

In [332]:
dep_del_no = departure_analysis_n.groupby('origin_airport').size().sort_values(ascending=False)
dep_del_no

origin_airport
ATL    214497
ORD    155956
DFW    137108
LAX    110595
DEN    104749
        ...  
ILG        42
ADK        34
AKN        33
GST        31
ITH        19
Length: 322, dtype: int64

In [341]:
prop_dep_del_y = dep_del_yes/flights_total
prop_dep_del_y.sort_values(ascending=False).head(20) #proportion a flight from origin airport will be early or on time 

origin_airport
GUM    0.679758
ADK    0.617978
GST    0.592105
ILG    0.557895
MVY    0.541463
DAL    0.525124
MDW    0.517453
BWI    0.494677
UST    0.493056
HOU    0.491067
AKN    0.476190
DEN    0.459867
PBG    0.455197
OAK    0.453992
EWR    0.452697
HYA    0.439024
ORD    0.437664
LAS    0.431003
IAD    0.430519
ACK    0.425926
dtype: float64

In [334]:
prop_dep_del_n = dep_del_no/flights_total
prop_dep_del_n.sort_values(ascending=False)

origin_airport
VEL    0.950000
CNY    0.946341
EKO    0.909091
LWS    0.908163
PIH    0.889571
         ...   
MVY    0.458537
ILG    0.442105
GST    0.407895
ADK    0.382022
GUM    0.320242
Length: 322, dtype: float64

Wow, airport VEL has a very good proportion of flights leaving on time or early! 

In [335]:
prop_dep_del_n['LAX']

0.5744926211242072

In [336]:
prop_dep_del_n['SAN']

0.6165912823024402

In [348]:
prop_dep_del_n['SJC']

0.6347651077803083

In [337]:
prop_dep_del_n['SFO']

0.597531879884821

In [342]:
prop_dep_del_y['SAN']

0.38340871769755985

In [343]:
prop_dep_del_y['LAX']

0.4255073788757928

In [349]:
prop_dep_del_y['SJC']

0.36523489221969174

In [344]:
prop_dep_del_y['SFO']

0.40246812011517896

All 4 of my airports are able to get planes back out of their gates reletively on time, SJC being the most efficient in general. 
These airports overall are fairly similar in their efficiencies, though it would seem the more air traffic you have indeed the higher chance your arrival or departure, or both, time will be impacted. I have decided that for my trip, in order to minimize any travel delays, I should fly from SAN -> SJC and return from SJC-> SAN. Based on the proportion of flights for all these airports, I have a nice 63% of arriving early into SAN and 61% of not being delayed leaving SAN. For SJC I have a 60% of arriving early or on time and a 63% chance of not expereincing any departure delay. I would say its the most important to consider the airports with the least amount of delarture delay. Human intuition and common sense tells me that the more likley an airport is to have a departure delay, the more likley that same gate will delay the arrival of the next plane and so on.

 I should avoid LAX in general due to both its high traffic volume and higher proportion percentage of arrive/departure delays. SFO has just a hair better proportion of flights arriving early, but when I consider that theyre a bit slower than SJC and getting planes back in the air, I think its best overall to use the two smaller airports though it would seem SFO is a fine option if needed or if money became a factor. 

SUMMARY:                    
In oder to answer the posed questions I needed to organize the data in a few different ways. To determine the number of flights delayed/not delayed I organized the arrival_delay column to represent flights >0 to respreent delayed and flights <=0 to represent on time or early. To proivide the proportion of delayed flights by an airport (i.e. the airport with the most amount of departing fights arriving late), I had to identify first the total number of flights delayed leaving a specific airport and divide that by the total number of flights leaving that specific airport. I indicated beneath the cell which airports had the most/least. I was able to show the probability of being delayed flying into any given airport, if you're flying into ATL theres a 2% you'll be delayed! This sounds low, but truly I am rarely delayed from my arrival time when I fly from most any aiport. (departure delay is a different story). I know there are 322 unique airports in this dataset and leaving from GST is a bad idea because they have the highest proportion of flights arriving late at their respective destination with 67%. 

I leanred that ATL is by a good margin the most trafficked airport. I believe I am correct when I say when flying out of ATL you have a 33% chance of arriving late to your destination, based off the proportion of flights going out of ATL that arrive late to their target destinations. You have a ~2% of arriving at your destination late when leaving/arriving at ATL based on the probability that any (5.2M) flight will have an arrival delay. Its quite intersting that the actual proportion is so much higher than the probability! In general the probability you will be on a flight that is delayed leaving or arriving at any aiport is about 37% based on the provided data. I was curious the chance you would be delayed on 25 out of 100 flights, based on the 37% chance your flight gets delayed. Bernoullis equation would say you have a 3.5% chace of experiencing 25 arrival delays from 100 unique flights. Good news if you're a frequent flyer, pretty good odds! 